In [1]:
from tensorflow.python.keras.datasets import mnist

C:\Users\User\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [11]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [9]:
# x_train[0]

In [10]:
# y_train[0]

In [14]:
# 입력 데이터를 네트워크에 맞게 변형

# 여기서는 2차원으로 변형하고 float형으로 변환한 후에
# 0부터 1 사이의 값을 가지도록 스케일 변환
x_train = x_train.reshape(60000, 784)
x_train = x_train/255.
x_test = x_test.reshape(10000, 784)
x_test = x_test/255.

In [18]:
# 클래스 레이블을 네트워크에 맞게 변형
# 정수가 저장된 클래스 레이블을 0, 1로 구성된 벡터로 변환 : 원핫(one-hot) 벡터
# to_categorical

from tensorflow.python.keras.utils import to_categorical

# 0, 1, 2, 3, 4, 5, 6, 7, 8, 9 → 10
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

#### Sequential API를 사용하여 다층 신경망 구축

- Sequential API에서는 준비된 레이어를 add 메서드로 추가하는 것만으로 간단히 모델 구축 가능

In [19]:
# 모델 구축 준비
from tensorflow.python.keras.models import Sequential

model = Sequential()

In [21]:
# 중간층 추가

# Dense 레이어를 이용해서 완전연결 계층 추가
# 완전연결 계층 : 모든 입력이 모든 뉴런과 결합된 층
# 입력에 가중치를 곱한 후 더한다
# 케라스에서는 Sequential API의 add 메서드를 사용해서 중간층 추가

from tensorflow.python.keras.layers import Dense

model.add(
    Dense(
        units = 64,             # 뉴런의 수 (출력 차원)
        input_shape=(784, ),    # 입력되는 텐서의 형태
        activation = 'relu'     # 활성화 함수의 종류
    )
)

Instructions for updating:
Colocations handled automatically by placer.


In [22]:
# 출력층 추가

# Dense 레이어를 한층 더 추가해서 출력층 추가
# MNIST 데이터는 0부터 9까지의 숫자 중에서 하나를 레이블로 가진 10 클래스 분류
# → 모델의 출력 차원을 10으로 지정

# 1층에서는 입력되는 텐서의 형태를 input_shape로 지정했지만,
# 2층 이후에는 케라스가 input_shape를 자동으로 계산해주므로 생략 가능

model.add(
    Dense(
        units = 10,
        activation = 'softmax'
    )
)

In [ ]:
# 교차엔트로피 : 두 개의 확률 분포 사이에 정의된 척도
# 분류 문제에서는 이 값이 작아지도록 학습
# 케라스에서는 이진 분류일 때는 binary_crossentropy
#              다중 분류일 때는 categorical_crossentropy

In [25]:
# 구축한 모델 학습

# 학습할 때 각 에폭에서 손실 함수의 값과 분류 정확도를 얻으려면 callbacks 모듈을 이용

# acc : 구축한 모델의 학습용 데이터에 대한 분류 정확도
# loss : (구축한 모델의) 손실 함수의 값
# val_acc : 구축한 모델의 검증용 데이터에 대한 분류 정확도
# val_loss : (구축한 모델의) 손실 함수의 값

# Adam을 이용한 모델의 MNIST 데이터 학습
from tensorflow.python.keras.callbacks import TensorBoard

model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)
tsb = TensorBoard(log_dir='./logs')
history_adam = model.fit(
    x_train,
    y_train,
    batch_size = 32,
    epochs = 20,
    validation_split = 0.2,
    callbacks = [tsb]
)

Train on 48000 samples, validate on 12000 samples
Epoch 1/20
48000/48000 [==============================] - 6s 117us/sample - loss: 0.1768 - acc: 0.9483 - val_loss: 0.1411 - val_acc: 0.9578
Epoch 2/20
48000/48000 [==============================] - 5s 106us/sample - loss: 0.1230 - acc: 0.9637 - val_loss: 0.1258 - val_acc: 0.9641
Epoch 3/20
48000/48000 [==============================] - 5s 96us/sample - loss: 0.0947 - acc: 0.9719 - val_loss: 0.1056 - val_acc: 0.9693
Epoch 4/20
48000/48000 [==============================] - 5s 98us/sample - loss: 0.0768 - acc: 0.9769 - val_loss: 0.1065 - val_acc: 0.9689
Epoch 5/20
48000/48000 [==============================] - 5s 107us/sample - loss: 0.0638 - acc: 0.9802 - val_loss: 0.1102 - val_acc: 0.9671
Epoch 6/20
48000/48000 [==============================] - 5s 94us/sample - loss: 0.0520 - acc: 0.9840 - val_loss: 0.0959 - val_acc: 0.9725
Epoch 7/20
48000/48000 [==============================] - 5s 100us/sample - loss: 0.0453 - acc: 0.9861 - val_loss

#### Functional API로 모델 구축

In [30]:
# Functional API를 이용한 모델 구축 준비
from tensorflow.python.keras.datasets import mnist
from tensorflow.python.keras.utils import to_categorical
from tensorflow.python.keras.callbacks import TensorBoard
from tensorflow.python.keras.layers import Input, Dense
from tensorflow.python.keras.models import Model

(x_train, y_train), (x_test, y_test) = mnist.load_data()
# x_train.shape, y_train.shape, x_test.shape, y_test.shape
x_train = x_train.reshape(60000, 784)
x_train = x_train/255.
x_test = x_test.reshape(10000, 784)
x_test = x_test/255.
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
tsb = TensorBoard(log_dir='./logs')

In [31]:
# Functional API로 모델 구축

input = Input( shape=(784, ) )
middle = Dense( units=64, activation='relu' )( input )
output = Dense( units=10, activation='softmax' )( middle )
model = Model( inputs=[input], outputs=[output] )

In [ ]:
# Functional API에서도 레이어 객체를 사용해서
# 텐서를 다음 층의 인수로 전달함으로써 모델을 구축할 수 있다

# Functional API에서는 model 클래스의 인수로
# inputs의 입력 텐서와 outputs의 출력 텐서를 지정하는 방법으로 모델 구축

# 구축한 모델은 Sequential API로 구축한 모델과 같아서
# 모델 컴파일과 학습으로
# compile과 fit 메서드 사용 가능

In [32]:
# 구축한 모델의 컴파일
model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [33]:
# MNIST 데이터 세트 학습
model.fit(
    x_train,
    y_train,
    batch_size = 32,
    epochs = 20,
    # 학습할 때 각 에폭에서 손실 함수의 값과 분류 정확도를 얻으려면 callbacks 모듈
    callbacks = [tsb],  
    validation_split=0.2
)

Train on 48000 samples, validate on 12000 samples
Epoch 1/20
48000/48000 [==============================] - 5s 109us/sample - loss: 0.3246 - acc: 0.9087 - val_loss: 0.1879 - val_acc: 0.9463
Epoch 2/20
48000/48000 [==============================] - 5s 104us/sample - loss: 0.1597 - acc: 0.9535 - val_loss: 0.1364 - val_acc: 0.9607
Epoch 3/20
48000/48000 [==============================] - 5s 97us/sample - loss: 0.1163 - acc: 0.9656 - val_loss: 0.1204 - val_acc: 0.9650
Epoch 4/20
48000/48000 [==============================] - 5s 104us/sample - loss: 0.0910 - acc: 0.9738 - val_loss: 0.1135 - val_acc: 0.9657
Epoch 5/20
48000/48000 [==============================] - 6s 117us/sample - loss: 0.0743 - acc: 0.9775 - val_loss: 0.1050 - val_acc: 0.9684
Epoch 6/20
48000/48000 [==============================] - 5s 100us/sample - loss: 0.0616 - acc: 0.9818 - val_loss: 0.1005 - val_acc: 0.9705
Epoch 7/20
48000/48000 [==============================] - 4s 93us/sample - loss: 0.0531 - acc: 0.9839 - val_los